In [2]:
# numpy and pandas imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# ML Libraries for scikit
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.feature_selection import SelectKBest, f_classif, chi2
from sklearn import tree
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,\
                            precision_recall_curve,precision_score,recall_score,roc_auc_score,roc_curve,\
                            matthews_corrcoef, f1_score, make_scorer, auc
from scipy.stats import skew

#  Recursive Feature Elimination
from sklearn.feature_selection import RFE
        
# Python imports
from math import log, sqrt
import re
import collections
import warnings

# decision tree visualization related imports
import pydotplus
import graphviz

from IPython.display import Image

warnings.filterwarnings('ignore')

pd.options.display.max_columns = 400

In [3]:
train =pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
df= train.copy()

In [4]:
train.head(10)

,Id,x0_t0,x0_t1,x0_t2,x0_t3,x0_t4,x0_t5,x0_t6,x1_t0,x1_t1,x1_t2,x1_t3,x1_t4,x1_t5,x1_t6,x2_t0,x2_t1,x2_t2,x2_t3,x2_t4,x2_t5,x2_t6,x3_t0,x3_t1,x3_t2,x3_t3,x3_t4,x3_t5,x3_t6,x4_t0,x4_t1,x4_t2,x4_t3,x4_t4,x4_t5,x4_t6,x5_t0,x5_t1,x5_t2,x5_t3,x5_t4,x5_t5,x5_t6,x6_t0,x6_t1,x6_t2,x6_t3,x6_t4,x6_t5,x6_t6,x7_t0,x7_t1,x7_t2,x7_t3,x7_t4,x7_t5,x7_t6,x8_t0,x8_t1,x8_t2,x8_t3,x8_t4,x8_t5,x8_t6,x9_t0,x9_t1,x9_t2,x9_t3,x9_t4,x9_t5,x9_t6,x10_t0,x10_t1,x10_t2,x10_t3,x10_t4,x10_t5,x10_t6,x11_t0,x11_t1,x11_t2,x11_t3,x11_t4,x11_t5,x11_t6,x12_t0,x12_t1,x12_t2,x12_t3,x12_t4,x12_t5,x12_t6,x13_t0,x13_t1,x13_t2,x13_t3,x13_t4,x13_t5,x13_t6,x14_t0,x14_t1,x14_t2,x14_t3,x14_t4,x14_t5,x14_t6,x15_t0,x15_t1,x15_t2,x15_t3,x15_t4,x15_t5,x15_t6,x16_t0,x16_t1,x16_t2,x16_t3,x16_t4,x16_t5,x16_t6,x17_t0,x17_t1,x17_t2,x17_t3,x17_t4,x17_t5,x17_t6,x18_t0,x18_t1,x18_t2,x18_t3,x18_t4,x18_t5,x18_t6,x19_t0,x19_t1,x19_t2,x19_t3,x19_t4,x19_t5,x19_t6,x20_t0,x20_t1,x20_t2,x20_t3,x20_t4,x20_t5,x20_t6,x21_t0,x21_t1,x21_t2,x21_t3,x21_t4,x21_t5,x21_t6,x22_t0,x22_t1,x22_t2,x22_t3,x22_t4,x22_t5,x22_t6,x23_t0,x23_t1,x23_t2,x23_t3,x23_t4,x23_t5,x23_t6,x24_t0,x24_t1,x24_t2,x24_t3,x24_t4,x24_t5,x24_t6,x25_t0,x25_t1,x25_t2,x25_t3,x25_t4,x25_t5,x25_t6,x26_t0,x26_t1,x26_t2,x26_t3,x26_t4,x26_t5,x26_t6,x27_t0,x27_t1,x27_t2,x27_t3,x27_t4,x27_t5,x27_t6,x28_t0,x28_t1,x28_t2,x28_t3,x28_t4,x28_t5,x28_t6,x29_t0,x29_t1,x29_t2,x29_t3,x29_t4,x29_t5,x29_t6,x30_t0,x30_t1,x30_t2,x30_t3,x30_t4,x30_t5,x30_t6,x32_t0,x32_t1,x32_t2,x32_t3,x32_t4,x32_t5,x32_t6,x31_t0,x31_t1,x31_t2,x31_t3,x31_t4,x31_t5,x31_t6,x34_t0,x34_t1,x34_t2,x34_t3,x34_t4,x34_t5,x34_t6,x33_t0,x33_t1,x33_t2,x33_t3,x33_t4,x33_t5,x33_t6,x36_t0,x36_t1,x36_t2,x36_t3,x36_t4,x36_t5,x36_t6,x35_t0,x35_t1,x35_t2,x35_t3,x35_t4,x35_t5,x35_t6,x37_t0,x37_t1,x37_t2,x37_t3,x37_t4,x37_t5,x37_t6,x38_t0,x38_t1,x38_t2,x38_t3,x38_t4,x38_t5,x38_t6,x39_t0,x39_t1,x39_t2,x39_t3,x39_t4,x39_t5,x39_t6,x40_t0,x40_t1,x40_t2,x40_t3,x40_t4,x40_t5,x40_t6,x41_t0,x41_t1,x41_t2,x41_t3,x41_t4,x41_t5,x41_t6,x42_t0,x42_t1,x42_t2,x42_t3,x42_t4,x42_t5,x42_t6,x43_t0,x43_t1,x43_t2,x43_t3,x43_t4,x43_t5,x43_t6,x44_t0,x44_t1,x44_t2,x44_t3,x44_t4,x44_t5,x44_t6,x45_t0,x45_t1,x45_t2,x45_t3,x45_t4,x45_t5,x45_t6,x46_t0,x46_t1,x46_t2,x46_t3,x46_t4,x46_t5,x46_t6,x47_t0,x47_t1,x47_t2,x47_t3,x47_t4,x47_t5,x47_t6,x48_t0,x48_t1,x48_t2,x48_t3,x48_t4,x48_t5,x48_t6,x49_t0,x49_t1,x49_t2,x49_t3,x49_t4,x49_t5,x49_t6,x50_t0,x50_t1,x50_t2,x50_t3,x50_t4,x50_t5,x50_t6,x51_t0,x51_t1,x51_t2,x51_t3,x51_t4,x51_t5,x51_t6,x52_t0,x52_t1,x52_t2,x52_t3,x52_t4,x52_t5,x52_t6,x53_t0,x53_t1,x53_t2,x53_t3,x53_t4,x53_t5,x53_t6,x54_t0,x54_t1,x54_t2,x54_t3,x54_t4,x54_t5,x54_t6,y2
0,706,762.212216,753.162754,754.610620,762.787135,768.170347,769.815486,750.950000,0.902534,0.873314,0.807268,0.862909,0.836029,0.902826,1.37600,1.0,1.0,1.0,1.0,1.0,1.0,1.0,21.262724,21.044805,21.449664,20.734400,21.135345,20.868361,27.900000,6.7,6.7,6.7,6.7,6.7,6.7,6.9,748.773616,745.635196,745.722604,747.266661,753.433793,752.713959,746.300000,788.799988,788.799988,788.799988,788.799988,788.799988,788.799988,795.830000,810.299988,810.299988,810.299988,810.299988,810.299988,810.299988,816.480000,4.4,4.4,4.4,4.4,4.4,4.4,4.4,0.243978,0.244055,0.245363,0.248112,0.335132,0.257364,0.310000,81.980046,82.087889,82.129742,82.093134,82.041982,82.008582,81.490000,15.277053,13.794279,16.751010,21.203234,24.425490,26.686207,9.400000,473.514571,473.415871,471.324834,472.325497,468.656301,469.211082,421.730000,460.856047,458.579524,454.148703,451.174712,446.458846,439.673132,495.530000,0.897013,0.897303,0.897223,0.897176,0.897156,0.897131,0.900000,1300.0,1300.0,1300.0,1300.000000,1300.0,1300.0,1704.25,65.0,65.0,65.0,65.0,65.0,65.0,55.66,702.238299,701.240283,696.300131,694.692963,701.764607,701.260836,697.240000,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,27.000000,27.000000,27.015366,27.00,27.000000,27.000000,27.0,751.197977,758.306068,768.134158,790.719635,784.526051,775.290000,775.29,0.407567,0.810007,0